## 파이콘 벡터 DB에 저장 및 검색이 가능하도록 설정

In [ ]:
# 0. 패키지/라이브러리 설치
# %pip install -qU docx2txt langchain_community langchain-text-splitters langchain_chroma
# %pip install langchain-pinecone
# %pip install pinecone

In [1]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고 분할 
loader = Docx2txtLoader('law_2.docx') # --> 인스턴스 생성

text_splitter = RecursiveCharacterTextSplitter( # --> 분할 설정
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

# 2. 임베딩 -> 백터 데이터베이스에 저장 
## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(
    model='text-embedding-3-large'
)

## 2.3 Pinecone: server
api_key = os.getenv('PINECONE_API_KEY')
pc = Pinecone(api_key=api_key)

c:\Users\2\miniconda3\envs\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
## 2.4 벡터 스토어에 저장(한번만 실행)
database = PineconeVectorStore.from_documents(
    index_name='law-index-2',
    embedding=embedding,
    documents=document_list,    
)

In [15]:
## 2.5 벡터 스토어에서 읽어오기
database = PineconeVectorStore(
    index_name='law-index-2',
    embedding=embedding,
)

# 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색  #############
## 3.1 사용자 질문
query = '전세사기 피해주택 임대인의 국세는 어떻게 계산하나요? 정확한 수식을 알려주세요'

## 3.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query, k=2)

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ###########
## 4.1 프롬프트 작성
prompt = '''
[identity]
- 당신은 전세사기 피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.
- 답변시 관련 법률 조항을 같이 출력하세요.
- 전세사기피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

Question: {query}
'''

## 4.2 프롬프트 변수에 값 설정(format 함수)
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 4.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI()

## 4.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message.content

'전세사기 피해주택 임대인의 국세는 다음과 같은 계산식을 사용하여 산정합니다:\n\n1. 상속세, 증여세 및 종합부동산세의 경우:\n   A × (B / C)\n\n   - A: 전세사기피해주택의 임대인이 체납한 고지 또는 신고 건별 국세 금액\n   - B: 전세사기피해주택의 가격\n   - C: 전세사기피해주택의 임대인이 보유한 모든 주택의 가격 합계액\n\n관련 법률 조항: 전세사기피해자 지원 및 주거안정에 관한 특별법 시행령 제3조제1항제1호가목.'